In [80]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize WebDriver
driver = webdriver.Chrome()

# Log in to Instagram before running this
driver.get("https://www.instagram.com/")

# Wait for login (ensure this part is working)
time.sleep(10)

# List to store extracted post data
post_data = []

# Loop through each post URL
for url in unique_post_urls:
    try:
        driver.get("https://www.instagram.com" + url)
        time.sleep(3)  # Allow content to load

        # Extract post caption
        caption = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//span[contains(@class, 'x1lliihq')]"))
        ).text

        # Extract image URL
        img_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//img[contains(@class, 'x5yr21d')]"))
        )
        img_url = img_element.get_attribute("src")

        # Append data
        post_data.append({"url": url, "caption": caption, "image": img_url})
        print(f"Scraped: {url}")

    except Exception as e:
        print(f"Error processing {url}: {e}")

# Print extracted data
print(post_data)
import os
import json
import requests
from urllib.parse import urlparse

# Ensure post_data is defined before use
post_data = post_data if 'post_data' in globals() else []

# Directory to save images
download_dir = "Instagram_Scrape"
image_dir = os.path.join(download_dir, "images")
os.makedirs(image_dir, exist_ok=True)

# Save data to a JSON file
json_file_path = os.path.join(download_dir, "instagram_data.json")
with open(json_file_path, "w", encoding="utf-8") as json_file:
    json.dump(post_data, json_file, ensure_ascii=False, indent=4)
print(f"Saved scraped data to {json_file_path}")

# Download images
for post in post_data:
    try:
        img_url = post.get("image")
        if not img_url:
            print(f"Skipping post with missing image: {post}")
            continue
        
        response = requests.get(img_url, stream=True)
        response.raise_for_status()

        # Extract filename from URL
        url_path = urlparse(img_url).path
        file_extension = os.path.splitext(url_path)[1] or ".jpg"
        file_name = f"{post.get('url', 'unknown').split('/')[-2]}{file_extension}"
        file_path = os.path.join(image_dir, file_name)

        # Save image
        with open(file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Downloaded: {file_path}")
    
    except requests.RequestException as e:
        print(f"Error downloading {img_url}: {e}")

print(f"All images downloaded to {image_dir}")



Scraped: /p/BpMgcILn_C0/
Scraped: /p/BfNt_qmFX40/
Scraped: /p/ByFAE7OhKbN/
Scraped: /p/BH5_bCoA3q_/
Scraped: /p/BxnMYZFjFKH/
Scraped: /p/BcyWFFpnEtZ/
Scraped: /p/CcYGE03JNVf/
Scraped: /p/Cbzfx2FhSZ5/
Scraped: /p/BC21e3-tyKV/
Scraped: /p/B9iVDvBggOX/
Scraped: /p/CL9TS_tBAud/
Scraped: /p/Btnq6ipFk1M/
Scraped: /p/BXFFK8Bl-uk/
Scraped: /p/C_oqHZXSJNe/
Scraped: /p/CnTGMa-o8VW/
Scraped: /p/CCTLgKLheMc/
Scraped: /p/B1GKcQ6hYDs/
Scraped: /p/CY31aKRI_kg/
Scraped: /p/BiLj5cPA3D4/
Scraped: /p/CZ0kLE2BYCH/
Scraped: /p/CWfaI1uPKkl/
Scraped: /p/BfUp_LGDvgD/
Scraped: /p/w40jPnimVf/
Scraped: /p/CnzVtJSBVhB/
Scraped: /p/CZWz6QgLfsx/
Scraped: /p/CJvEU6vnACo/
Scraped: /p/Cst95qGLW-l/
Scraped: /p/CYg-mvUBGJN/
Scraped: /p/BgviIWHH29i/
Scraped: /p/_9u00BEQWw/
Scraped: /p/_fX0kUjIV8/
Scraped: /p/COKAeH6Fbq4/
Scraped: /p/BaQ1GOMBL8d/
Scraped: /p/Btnq-08F--M/
Scraped: /p/BkpuvXInFAa/
Scraped: /p/B_KrxYoJ_-n/
Scraped: /p/CLw78eMl2EL/
Scraped: /p/BWZaq2OB5x8/
Scraped: /p/BJFYQxXBAII/
Scraped: /p/C-Pun0tp9mA/
Scr

In [82]:
import os
import json
import pandas as pd

# Ensure required module is installed
try:
    import openpyxl
except ModuleNotFoundError:
    import subprocess
    subprocess.check_call(["python", "-m", "pip", "install", "openpyxl"])
    import openpyxl

# Directory to save extracted data
download_dir = "Instagram_Scrape"
os.makedirs(download_dir, exist_ok=True)

# Ensure post_data is initialized properly before using it
if 'post_data' not in locals():
    post_data = []

# Convert data to DataFrame if post_data is not empty
if post_data:
    df = pd.DataFrame(post_data)
    
    # Ensure 'caption' column exists before extracting hashtags
    if 'caption' in df.columns:
        df["hashtags"] = df["caption"].apply(lambda x: " ".join([word for word in x.split() if word.startswith("#")]) if isinstance(x, str) else "")
    else:
        df["hashtags"] = ""
else:
    df = pd.DataFrame(columns=["url", "caption", "image", "hashtags"])

# Save data to an Excel file
excel_file_path = os.path.join(download_dir, "instagram_data.xlsx")
df.to_excel(excel_file_path, index=False)
print(f"Saved scraped data to {excel_file_path}")


Saved scraped data to Instagram_Scrape\instagram_data.xlsx
